Based on a paper of F. Barlat

In [ ]:
%pylab inline
import time
import lib;reload(lib)
from lib import *
import yf;reload(yf)
from yf import *
import func_hard; reload(func_hard)
from func_hard import *
from MP import progress_bar
uet = progress_bar.update_elapsed_time
import multiprocessing
from multiprocessing import Pool
from scipy import optimize

In [ ]:
xdata,ydata=np.loadtxt('/Users/yj/Documents/ExpDat/IFSteel/Bulge/EXP_BULGE_JINKIM.txt').T
hparams,dum=optimize.curve_fit(f=func_swift,xdata=xdata,ydata=ydata)
eps_bar = np.linspace(0,1.,100)
sig_bar = func_swift(eps_bar,*hparams)
plot(eps_bar,sig_bar)
plot(xdata[::20],ydata[::20],'rx')
print hparams

In [ ]:
## Material card
func_har = func_swift
func_ptn = VonMises
func_yld = VonMises ## Associated Flow Rule
eqv_eps  = VonMisesE
##

sigma_bar = lambda eps: func_har(eps,*hparams)
eps_bar   = np.linspace(0,1.5,1000)

#fig=plt.figure(figsize=(3.5,3));ax=gca()
#ax.plot(eps_bar,sigma_bar(eps_bar))
#ax.set_xlabel(r'Strain $\bar{\varepsilon}$')
#ax.set_ylabel(r'Stress $\bar{\sigma}$')

# region A simulations - fairly straight forward.
#d_time = 1e-4 ## time increment.
#sr     = 1e-3 ## major component
delt_ebar = 1e-3 ## incremental step for the hardening curve.
alpha  = 1.0   ## BB
beta   = 0.0   ## No shear

## at the time=0?
eps_mx = 0.05
eps6=np.zeros(6)
ebar = 0.
fig=plt.figure(figsize=(11,3))
ax1=fig.add_subplot(131)
ax2=fig.add_subplot(132)
ax3=fig.add_subplot(133)
i=0
while (ebar<eps_mx):
    ebar      = ebar + delt_ebar
    sig_bar   = sigma_bar(ebar)
    ax3.plot(ebar,sig_bar,'k.')
    delt_work = delt_ebar * sig_bar
    
    ## Current stress tensor "direction"
    sig_norm6 = alph2sig6(alpha,beta)
    sig_norm6 = sig_norm6/func_yld(sig_norm6) ## equivalent stress state
    sig6      = sig_norm6*sig_bar

    ## Current strain rate tensor direction.
    deps6 = alph2eps(alpha,beta,func_ptn)  
    ## Correct deps6 magnitude by conjugating the incremental work
    dw = 0.
    for j in xrange(3):
        dw = dw + deps6[j]*sig6[j]
        dw = dw + 2*deps6[j+3]*sig6[j+3]
    x     = delt_work/dw
    deps6 = deps6*x
    eps6  = eps6 + deps6

    ax1.plot(sig6[0],sig6[1],'k.')
    ax2.plot(eps6[0],eps6[1],'k.')
    
    if i==0: ## draw yield locus
        loc1,loc2=y_locus(100,func_yld)
        loc1=loc1*sig_bar
        ax1.plot(loc1[0],loc1[1])
    i=i+1

ys_temp(ax1)
es_temp(ax2)
ss_temp(ax3)
ax2.grid('on')
ax1.set_xlim(0.,)
ax1.set_ylim(0.,)
fig.tight_layout()

fig.savefig('mk_algorithm.pdf',bbox_inches='tight')

In [ ]:
## Functionize the above
def FLD_A_one_path(alpha=1.0,beta=0.,
                   func_har = func_swift,
                   func_yld = VonMises,
                   func_hard = None,
                   ebar_mx=0.1,
                   delt_ebar=1e-4,
                   **yld_kwargs):
    """
    """
    eps6=np.zeros(6)
    ebar=0.
    i=0
    strain=[]
    stress=[]
    strain_eq=[]
    stress_eq=[]
    while (ebar<ebar_mx):
        ebar      = ebar + delt_ebar
        sig_bar   = func_hard(ebar)
        strain_eq.append(ebar)
        stress_eq.append(sig_bar)
        delt_work = delt_ebar * sig_bar
    
        if i==0: ## when initial anisotropy is considered (constant shape Yield Surface)
            ## Stress vector direction
            sig_norm6 = alph2sig6(alpha,beta)
            sig_norm6 = sig_norm6/func_yld(sig_norm6,**yld_kwargs) ## equivalent stress state
            ## Strains vector direction.
            deps6 = alph2eps(alpha,beta,func_yld,**yld_kwargs)  
        
        sig6      = sig_norm6*sig_bar
        ## Correct deps6 magnitude by conjugating the incremental work
        dw = 0.
        for j in xrange(3):
            dw = dw + deps6[j]*sig6[j]
            dw = dw + 2*deps6[j+3]*sig6[j+3]
        
        x     = delt_work/dw
        temp  = deps6*x
        eps6  = eps6 + temp
        
        i=i+1
        
        strain.append(eps6)
        stress.append(sig6)
        
    return np.array(strain),np.array(stress),np.array(strain_eq),np.array(stress_eq)

# Check different types of yield functions 

Von Mises

In [ ]:
sigma_bar = lambda eps: func_har(eps,*hparams)
alphs=np.linspace(0.,2.,10)

fig=plt.figure(figsize=(10,3))
ax1=fig.add_subplot(131)
ax2=fig.add_subplot(132)
ax3=fig.add_subplot(133)
for i in xrange(len(alphs)):
    eps6,sig6,ebar,sbar = FLD_A_one_path(
        alpha=alphs[i],beta=0.,
        func_har = func_swift,
        func_yld = VonMises,
        func_hard = sigma_bar,
        ebar_mx=0.1,
        delt_ebar=1e-4)

    ax1.plot(eps6[:,1],eps6[:,0],'r-')
    ax2.plot(sig6[:,1],sig6[:,0],'b-')
    ax3.plot(ebar,sbar)
    
ax2.set_xlim(-50,)
ax2.set_ylim(-50.,)
ys_tempr(ax1)
es_tempr(ax2)
fig.tight_layout()

Hosford

In [ ]:
sigma_bar = lambda eps: func_har(eps,*hparams)
alphs=np.linspace(0.,2.,10)

fig=plt.figure(figsize=(7,3))
ax1=fig.add_subplot(121)
ax2=fig.add_subplot(122)
for i in xrange(len(alphs)):
    eps6,sig6,ebar,sbar = FLD_A_one_path(
        alpha=alphs[i],beta=0.,
        func_har = func_swift,
        func_yld = Hosford,
        func_hard = sigma_bar,
        ebar_mx=0.1,
        delt_ebar=1e-4,a=6.)

    ax1.plot(eps6[:,1],eps6[:,0],'r-')
    ax2.plot(sig6[:,1],sig6[:,0],'b-')
    
ax2.set_xlim(-50,)
ax2.set_ylim(-50.,)
ys_tempr(ax1)
es_tempr(ax2)
fig.tight_layout()

Quadratic Hill

In [ ]:
sigma_bar = lambda eps: func_har(eps,*hparams)
alphs=np.linspace(0.,2.,10)

t0=time.time()

fig=plt.figure(figsize=(11,3))
ax1=fig.add_subplot(131)
ax2=fig.add_subplot(132)
ax3=fig.add_subplot(133)
for i in xrange(len(alphs)):
    eps6,sig6,ebar,sbar = FLD_A_one_path(
        alpha=alphs[i],beta=0.,
        func_har = func_swift,
        func_yld = QuadHill,
        func_hard = sigma_bar,
        ebar_mx=1.0,
        delt_ebar=1e-4,
        R0=0.5, R90=2.)

    ax1.plot(eps6[:,1],eps6[:,0],'r-')
    ax2.plot(sig6[:,1],sig6[:,0],'b-')
    ax3.plot(ebar,sbar)

uet(time.time()-t0,head='Elapsed time for Region A probing')

ax2.set_xlim(-50.,); ax2.set_ylim(-50.,)
ys_tempr(ax1); es_tempr(ax2)
fig.tight_layout()